#### external libs

In [ ]:
# libs
import os
import sys
import pickle as pk
from datetime import datetime, timedelta
import json
import pandas as pd
import numpy as np

sys.path.insert(0, "../")

from autotrend.util import (get_df_from_yf_historic_data_dump, 
                            estimate_volatility, 
                            linear_model_pred, 
                            draw_trendlines)
from autotrend.adaptive import trendlines_adaptive
from autotrend.batch import (find_best_num_lines, 
                             fit_pwl_model,
                             get_lines_boundries)

#
%config IPCompleter.greedy=True
%matplotlib inline

# 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#
def get_draw_trendlines_adaptive(data,
                                 price_column='adjclose',
                                 last_ind=0,
                                 incr=1,
                                 min_break_len=14,
                                 min_points=60,
                                 mv_ave_window=24,
                                 symmetric=True,
                                 alpha=0.5,
                                 title=''):

    ts, prices = list(data.index), list(data[price_column])
    x, y = list(range(len(ts))), prices

    range_estimate = estimate_volatility(
        y=prices[:min_points], mv_ave_window=mv_ave_window)

    splits, slopes_intercepts, biases = trendlines_adaptive(y=y,
                                                            last_ind=last_ind,
                                                            incr=incr,
                                                            min_break_len=min_break_len,
                                                            symmetric=symmetric,
                                                            max_range=range_estimate,
                                                            min_points=min_points,
                                                            alpha=alpha)

    lines, lower_lines, upper_lines = [], [], []

    i = 0
    for j, (a, b), (bias_lower, bias_upper) in zip(splits[1:], slopes_intercepts, biases):
        x = list(range(i, j))
        lines.append((x, [linear_model_pred(k-i, a, b) for k in range(i, j)]))
        lower_lines.append(
            (x, [linear_model_pred(k-i, a, b - bias_lower) for k in range(i, j)]))
        upper_lines.append(
            (x, [linear_model_pred(k-i, a, b + bias_upper) for k in range(i, j)]))
        i = j

    draw_trendlines(ts, prices, lines, lower_lines, upper_lines, title=title)

    return splits, slopes_intercepts, lines, lower_lines, upper_lines


def get_draw_trendlines_batch(data,
                              price_column='adjclose',
                              min_split_len=10,
                              min_num_lines=1,
                              max_num_lines=30,
                              mv_ave_window=24,
                              title=''):

    ts, prices = list(data.index), list(data[price_column])
    x, y = list(range(len(ts))), prices

    optimums, best_num_lines = find_best_num_lines(x,
                                                   y,
                                                   min_split_len,
                                                   min_num_lines,
                                                   max_num_lines,
                                                   mv_ave_window)

    optimum_d = [d for d in optimums if d["num_lines"] == best_num_lines][0]
    optimum = optimum_d['optimum']

    splits = x[:1] + optimum.x.tolist()
    splits.sort()

    params, A, ssr = fit_pwl_model(x, y, splits)
    y_hat = np.matmul(A, params)
    lines, lower_lines, upper_lines = get_lines_boundries(x, 
                                                          y_hat, 
                                                          prices, 
                                                          splits, 
                                                          issorted=True)

    draw_trendlines(ts, prices, lines, lower_lines, upper_lines, title=title)

    return splits, optimum_d, lines, lower_lines, upper_lines


In [ ]:
# get data dump from Yahoo Finance API
interval = "1d"
days = 5 * 365
today = datetime.today()
today_ts = int(today.timestamp())
start_ts = int((today - timedelta(days=days)).timestamp())

tickers = 'GE AAPL MSFT'.split()
tickers_file = '../resources/tickers.txt'
open(tickers_file, 'w').writelines("\n".join(tickers))
os.system(f"bash ../scripts/bash/get_historic_data.sh {tickers_file} {start_ts} {today_ts} {interval} ../resources/")


dfs_prices = {}
for ticker in tickers:
    dump = json.load(open(f"/tmp/{ticker}"))
    dfs_prices[ticker] = get_df_from_yf_historic_data_dump(dump)


In [ ]:
for ticker in tickers:
    data = dfs_prices[ticker]
    splits, slopes_intercepts, lines, lower_lines, upper_lines = get_draw_trendlines_adaptive(data,
                                                                                              price_column='adjclose',
                                                                                              last_ind=0,
                                                                                              incr=1,
                                                                                              min_break_len=14,
                                                                                              min_points=60,
                                                                                              mv_ave_window=100,
                                                                                              symmetric=True,
                                                                                              alpha=0.5,
                                                                                              title=f"{ticker} (adaptive)")
